In [29]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.preprocessing.image import ImageDataGenerator

import numpy as np
import matplotlib.pyplot as plt

from os import listdir
from os import system
import os
import random

import imageio

In [36]:
patch_size = 56
batch_size = 128

In [47]:
base_dir = '/scratch/image_datasets/2_for_learned_brief/ready'

train_data_dir      = base_dir + '/train'
validation_data_dir = base_dir + '/validation'
test_data_dir       = base_dir + '/test'

In [48]:
def loading_data(dir_patches):
    """Load all the patches from dir_patches into tensors for training the autoencoder.
    Return:
        patches  -- tensor of stacked patches    
    """
    files_patches = listdir(dir_patches + '/class0')
    files_patches.sort()
    
    patches = []
    
    for file_patch in files_patches:
        patch = imageio.imread(dir_patches + '/class0/' + file_patch)
        
        patches.append(patch)
        
    patches = np.array(patches)
    patches = patches.astype(np.float64) / 255
    patches = np.expand_dims(patches, -1)
        
    print("patches shape:", patches.shape)
    
    return patches

In [49]:
x_train = loading_data(train_data_dir)
x_validation = loading_data(validation_data_dir)

patches shape: (64598, 56, 56, 1)
patches shape: (8075, 56, 56, 1)


In [50]:
class Sampling(layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""

    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon

In [51]:
latent_dim = 128

encoder_inputs = keras.Input(shape=(patch_size, patch_size, 1))
x = layers.Conv2D(32, 3, activation="relu", strides=2, padding="same")(encoder_inputs)
x = layers.Conv2D(64, 3, activation="relu", strides=2, padding="same")(x)
x = layers.Conv2D(64, 3, activation="relu", strides=2, padding="same")(x)
x = layers.Flatten()(x)
x = layers.Dense(256, activation="relu")(x)
z_mean = layers.Dense(latent_dim, name="z_mean")(x)
z_log_var = layers.Dense(latent_dim, name="z_log_var")(x)
z = Sampling()([z_mean, z_log_var])
encoder = keras.Model(encoder_inputs, [z_mean, z_log_var, z], name="encoder")
encoder.summary()

Model: "encoder"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            [(None, 56, 56, 1)]  0                                            
__________________________________________________________________________________________________
conv2d_6 (Conv2D)               (None, 28, 28, 32)   320         input_6[0][0]                    
__________________________________________________________________________________________________
conv2d_7 (Conv2D)               (None, 14, 14, 64)   18496       conv2d_6[0][0]                   
__________________________________________________________________________________________________
conv2d_8 (Conv2D)               (None, 7, 7, 64)     36928       conv2d_7[0][0]                   
____________________________________________________________________________________________

In [52]:
latent_inputs = keras.Input(shape=(latent_dim,))
x = layers.Dense(7 * 7 * 64, activation="relu")(latent_inputs)
x = layers.Reshape((7, 7, 64))(x)
x = layers.Conv2DTranspose(64, 3, activation="relu", strides=2, padding="same")(x)
x = layers.Conv2DTranspose(64, 3, activation="relu", strides=2, padding="same")(x)
x = layers.Conv2DTranspose(32, 3, activation="relu", strides=2, padding="same")(x)
decoder_outputs = layers.Conv2DTranspose(1, 3, activation="sigmoid", padding="same")(x)
decoder = keras.Model(latent_inputs, decoder_outputs, name="decoder")
decoder.summary()

Model: "decoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         [(None, 128)]             0         
_________________________________________________________________
dense_6 (Dense)              (None, 3136)              404544    
_________________________________________________________________
reshape_3 (Reshape)          (None, 7, 7, 64)          0         
_________________________________________________________________
conv2d_transpose_12 (Conv2DT (None, 14, 14, 64)        36928     
_________________________________________________________________
conv2d_transpose_13 (Conv2DT (None, 28, 28, 64)        36928     
_________________________________________________________________
conv2d_transpose_14 (Conv2DT (None, 56, 56, 32)        18464     
_________________________________________________________________
conv2d_transpose_15 (Conv2DT (None, 56, 56, 1)         289 

In [53]:
class VAE(keras.Model):
    def __init__(self, encoder, decoder, **kwargs):
        super(VAE, self).__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder

    def train_step(self, data):
        if isinstance(data, tuple):
            data = data[0]
        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = encoder(data)
            reconstruction = decoder(z)
            reconstruction_loss = tf.reduce_mean(
                keras.losses.binary_crossentropy(data, reconstruction)
            )
            reconstruction_loss *= 28 * 28
            kl_loss = 1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var)
            kl_loss = tf.reduce_mean(kl_loss)
            kl_loss *= -0.5
            total_loss = reconstruction_loss + kl_loss
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        return {
            "loss": total_loss,
            "reconstruction_loss": reconstruction_loss,
            "kl_loss": kl_loss,
        }

In [30]:
image_datagen = ImageDataGenerator()

In [ ]:
vae = VAE(encoder, decoder)
vae.compile(optimizer=keras.optimizers.Adam())
vae.fit(x_train, epochs=50, batch_size=batch_size)


model_version = "0.0.0.1"
vae.save_weights(base_dir + '/vae_' + model_version + '.h5')


In [39]:
dataset = tf.data.Dataset.from_tensor_slices(x_train)


In [45]:
vae = VAE(encoder, decoder)
vae.compile(optimizer=keras.optimizers.Adam())

# vae.fit(x_train, epochs=30, batch_size=128)
# vae.fit_generator(image_datagen.flow(x_train, x_train, batch_size=batch_size), steps_per_epoch=x_train.shape[0], epochs=50)
vae.fit(x_train, steps_per_epoch=x_train.shape[0], epochs=50, batch_size=batch_size)

model_version = "0.0.0.1"
vae.save_weights(base_dir + '/vae_' + model_version + '.h5')


Epoch 1/50
200/504 [==========>...................] - 2s 11ms/step - loss: 460.3369 - reconstruction_loss: 458.8441 - kl_loss: 1.4929


In [54]:
vae = VAE(encoder, decoder)
vae.compile(optimizer=keras.optimizers.Adam())

# vae.fit(x_train, epochs=30, batch_size=128)
# vae.fit_generator(image_datagen.flow(x_train, x_train, batch_size=batch_size), steps_per_epoch=x_train.shape[0], epochs=50)
vae.fit(x_train, epochs=50, batch_size=batch_size)

model_version = "0.0.0.1"
vae.save_weights(base_dir + '/vae_' + model_version + '.h5')


Epoch 1/50
505/505 [==============================] - 5s 11ms/step - loss: 460.6512 - reconstruction_loss: 459.5046 - kl_loss: 1.1465
Epoch 2/50
505/505 [==============================] - 5s 11ms/step - loss: 426.6239 - reconstruction_loss: 424.8351 - kl_loss: 1.7887
Epoch 3/50
505/505 [==============================] - 5s 11ms/step - loss: 423.1027 - reconstruction_loss: 421.3390 - kl_loss: 1.7637
Epoch 4/50
505/505 [==============================] - 5s 11ms/step - loss: 421.4314 - reconstruction_loss: 419.6671 - kl_loss: 1.7643
Epoch 5/50
505/505 [==============================] - 5s 11ms/step - loss: 420.3955 - reconstruction_loss: 418.6388 - kl_loss: 1.7567
Epoch 6/50
505/505 [==============================] - 5s 11ms/step - loss: 419.7329 - reconstruction_loss: 417.9953 - kl_loss: 1.7376
Epoch 7/50
505/505 [==============================] - 5s 11ms/step - loss: 419.2359 - reconstruction_loss: 417.5242 - kl_loss: 1.7117
Epoch 8/50
505/505 [==============================] - 5s 11ms/